# SVM Soft Margin Extension with NumPy

In [1]:
import numpy as np
import csv
import math
from numpy import genfromtxt
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
%matplotlib inline

## 1. MNIST

### Homework implementation

In [2]:
digits=load_digits()
X = digits.data
y = digits.target

# Scale training features
X_scale = StandardScaler()
X = X_scale.fit_transform(digits.data)

In [3]:
# Assign X and y the subset of data that describe the numbers 8 and 9

new_X = []
new_y = []
for i in range(len(X)):
    if y[i] == 8:
        new_X.append(X[i])
        new_y.append(y[i])
    elif y[i] == 9:
        new_X.append(X[i])
        new_y.append(y[i])
new_X = np.array(new_X)
new_y = np.array(new_y)

X = new_X
y = new_y

In [4]:
# Train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6,random_state=42)


In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(141, 64)
(141,)
(213, 64)
(213,)


In [6]:
y_train.shape

(141,)

In [7]:
def kernel_svm(X, y): 

    m,n = X.shape
    y = y.reshape(-1,1)
    X_y = X*y
    H = np.dot(X_y, X_y.T)
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    G = matrix(-np.eye(m))
    h = matrix(np.zeros(m))
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))
    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_svm(X_train, y_train)

     pcost       dcost       gap    pres   dres
 0: -3.8262e-02 -4.0188e-02  2e+02  1e+01  1e+00
 1: -3.0905e-04 -1.1130e-04  2e+00  1e-01  1e-02
 2: -1.8827e-05 -6.7504e-05  3e-02  2e-03  2e-04
 3: -6.8873e-06 -1.1057e-05  1e-03  8e-05  7e-06
 4: -1.2441e-06 -1.0286e-07  3e-05  3e-06  2e-07
 5: -1.3608e-08 -1.0581e-11  4e-07  3e-08  2e-09
 6: -1.3618e-10 -1.0581e-15  4e-09  3e-10  2e-11
Optimal solution found.


In [8]:
def kernel_svm(X, y): 

    m,n = X.shape
    y = y.reshape(-1,1)*1.
    X_y = X*y
    H = np.dot(X_y, X_y.T)
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    G = matrix(-np.eye(m))
    h = matrix(np.zeros(m))
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))
    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_svm(X_train, y_train)
print(alphas)

     pcost       dcost       gap    pres   dres
 0: -3.8262e-02 -4.0188e-02  2e+02  1e+01  1e+00
 1: -3.0905e-04 -1.1130e-04  2e+00  1e-01  1e-02
 2: -1.8827e-05 -6.7504e-05  3e-02  2e-03  2e-04
 3: -6.8873e-06 -1.1057e-05  1e-03  8e-05  7e-06
 4: -1.2441e-06 -1.0286e-07  3e-05  3e-06  2e-07
 5: -1.3608e-08 -1.0581e-11  4e-07  3e-08  2e-09
 6: -1.3618e-10 -1.0581e-15  4e-09  3e-10  2e-11
Optimal solution found.
[ 1.10008300e-11  2.09343171e-11 -1.52888491e-11 -1.67481751e-11
 -1.66363190e-11 -1.37071990e-11 -1.49454228e-11 -4.22863306e-12
 -1.65478139e-11  3.43433702e-11  2.60853523e-11  2.11618856e-11
 -1.54137825e-11 -1.85374511e-11 -1.44943188e-11  2.11612318e-11
  2.58213534e-11 -1.36190907e-11 -1.72111703e-11 -1.74237986e-11
  1.34051841e-11  8.87109839e-12 -1.49446968e-11 -1.58828641e-11
 -1.29457904e-11  1.39840867e-11  2.43037776e-11  2.68312148e-11
  1.28495891e-11 -9.40838869e-12 -1.56115124e-11  2.20192537e-11
  2.57802252e-11 -1.64524268e-11  1.20248647e-11  8.71851732e-12


In [9]:
def compute_classification_boundary (X, y, alpha):
    #cond = (alphas > 1e-3).reshape(-1)
    cond = [i for i in range(len(alphas)) if alphas[i] > 1e-12]
    w = np.dot(X.T, alpha*y).reshape(-1,1)
    w0 = y[cond] - np.dot(X[cond], w)
    w0 = np.mean(w0)
    return w, w0



w, w0 = compute_classification_boundary(X_train, y_train, alphas)

In [10]:
# Determine which training examples are support vectors
support_vectors = []

for i in range(len(alphas)):
    if alphas[i] > 1e-12:
        support_vectors.append([X_train[i], y_train[i], i])

In [11]:
def K(xi, xj):
    return np.dot(xi,xj)

alpha_indices = [support_vectors[i][2] for i in range(len(support_vectors))]
print(alpha_indices)

[0, 1, 9, 10, 11, 15, 16, 20, 21, 25, 26, 27, 28, 31, 32, 34, 35, 40, 45, 46, 50, 53, 54, 55, 57, 58, 60, 61, 63, 64, 65, 67, 69, 73, 75, 76, 79, 83, 86, 89, 91, 95, 96, 103, 104, 107, 108, 110, 114, 117, 118, 120, 121, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140]


In [12]:
def f_dual(x):
    summation = 0
    for i in range(len(support_vectors)):
        summation += alphas[alpha_indices[i]]*y_train[alpha_indices[i]]*K(X_train[alpha_indices[i]],x)
    if (summation >= 0):
        return 8
    else:
        return 9

In [13]:
# Test SVM dual classifier on X_test

def predict(X):
    predictions = []
    for i in range(len(X_test)):
        predictions.append(f_dual(X_test[i]))
    return predictions

y_pred = predict(X_test)

In [14]:
# Print accuracy

print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy is 79.81220657276995%


### Extended implementation using Numpy

In [23]:
def kernel_soft_margin_svm(X, y, C): 

    m,n = X.shape
    y = y.reshape(-1,1)
    X_y = X*y
    H = np.dot(X_y, X_y.T)
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    
    # Changed G and h
    G = matrix(np.vstack((np.diag(np.ones(m))*-1, np.identity(m))))
    h = matrix(np.hstack((np.zeros(m), np.ones(m)*C)))
    
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))
    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_soft_margin_svm(X_train, y_train, 0.1)

print(alphas)

     pcost       dcost       gap    pres   dres
 0: -2.1056e-02 -1.4523e+01  3e+02  1e+01  3e-14
 1: -3.6794e-03 -1.2104e+01  2e+01  2e-01  3e-14
 2: -1.3539e-05 -4.4261e-01  5e-01  4e-03  5e-15
 3: -6.7445e-06 -1.3705e-02  1e-02  1e-04  6e-16
 4: -1.4392e-06 -5.3602e-04  6e-04  4e-06  5e-16
 5: -1.6065e-08 -5.4504e-06  6e-06  4e-08  8e-16
 6: -1.6080e-10 -5.4504e-08  6e-08  4e-10  8e-16
Optimal solution found.
[ 1.36585483e-11  2.27506384e-11 -1.92366639e-11 -1.98397047e-11
 -2.00293069e-11 -1.61368576e-11 -1.81284643e-11  2.00850294e-12
 -2.03122376e-11  4.36445838e-11  2.90944943e-11  2.33602023e-11
 -1.95539561e-11 -2.12733573e-11 -1.69055886e-11  2.44005157e-11
  2.84644718e-11 -1.60393180e-11 -2.08773643e-11 -2.05332723e-11
  1.58030199e-11  1.03761550e-11 -1.75146329e-11 -1.81462117e-11
 -1.64446170e-11  1.71012765e-11  2.81641735e-11  3.21843997e-11
  1.55842114e-11 -1.12318390e-11 -1.92758127e-11  2.37963252e-11
  3.03680369e-11 -1.87172034e-11  1.57760173e-11  1.12448786e-11


In [17]:
def compute_classification_boundary (X, y, alpha):
    #cond = (alphas > 1e-3).reshape(-1)
    cond = [i for i in range(len(alphas)) if alphas[i] > 1e-12]
    w = np.dot(X.T, alpha*y).reshape(-1,1)
    w0 = y[cond] - np.dot(X[cond], w)
    w0 = np.mean(w0)
    return w, w0



w, w0 = compute_classification_boundary(X_train, y_train, alphas)

In [18]:
# Determine which training examples are support vectors
support_vectors = []

for i in range(len(alphas)):
    if alphas[i] > 1e-12:
        support_vectors.append([X_train[i], y_train[i], i])

# print("The following are support vectors: ")
# for i in range(len(support_vectors)):
#     print(support_vectors[i][0])

In [19]:
def K(xi, xj):
    return np.dot(xi,xj)

alpha_indices = [support_vectors[i][2] for i in range(len(support_vectors))]
print(alpha_indices)

[0, 1, 7, 9, 10, 11, 15, 16, 20, 21, 25, 26, 27, 28, 31, 32, 34, 35, 40, 45, 46, 50, 53, 54, 55, 57, 58, 60, 61, 63, 64, 65, 67, 69, 73, 75, 76, 79, 83, 86, 89, 91, 95, 96, 103, 104, 107, 108, 110, 114, 117, 118, 120, 121, 125, 126, 127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140]


In [20]:
def f_dual(x):
    summation = 0
    for i in range(len(support_vectors)):
        summation += alphas[alpha_indices[i]]*y_train[alpha_indices[i]]*K(X_train[alpha_indices[i]],x)
    if (summation >= 0):
        return 8
    else:
        return 9

In [21]:
# Test SVM dual classifier on X_test

def predict(X):
    predictions = []
    for i in range(len(X_test)):
        predictions.append(f_dual(X_test[i]))
    return predictions

y_pred = predict(X_test)

In [22]:
# Print accuracy

print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy is 80.28169014084507%


## 2. Fashion-MNIST

### Homework implementation

In [24]:
from keras.datasets import fashion_mnist
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

Using TensorFlow backend.


In [25]:
X_train = trainX
y_train = trainY
X_test = testX
y_test = testY

In [26]:
# Assign X_train and y_train the subset of data that describe the labels 0 and 2 (T-shirts and pullovers, respectively)

new_X_train = []
new_y_train = []
for i in range(len(X_train)):
    if y_train[i] == 0:
        new_X_train.append(X_train[i])
        new_y_train.append(y_train[i])
    elif y_train[i] == 2:
        new_X_train.append(X_train[i])
        new_y_train.append(y_train[i])
new_X_train = np.array(new_X_train)
new_y_train = np.array(new_y_train)

X_train = new_X_train
y_train = new_y_train

In [27]:
# Assign X_test and y_test the subset of data that describe the labels 0 and 2 (T-shirts and pullovers, respectively)

new_X_test = []
new_y_test = []
for i in range(len(X_test)):
    if y_test[i] == 0:
        new_X_test.append(X_test[i])
        new_y_test.append(y_test[i])
    elif y_test[i] == 2:
        new_X_test.append(X_test[i])
        new_y_test.append(y_test[i])
new_X_test = np.array(new_X_test)
new_y_test = np.array(new_y_test)

X_test = new_X_test
y_test = new_y_test

In [28]:
X_train = np.array([X_train[i].flatten() for i in range(len(X_train))])
X_test = np.array([X_test[i].flatten() for i in range(len(X_test))])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(12000, 784)
(12000,)
(2000, 784)
(2000,)


In [29]:
# Downsample the data

# Add y_train back as an additional column to X_train
y_train = y_train.reshape((-1,1))
X_train = np.append(X_train, y_train, axis=1)

# Add y_test back as an additional column to X_test
y_test = y_test.reshape((-1,1))
X_test = np.append(X_test, y_test, axis=1)

# Shuffle the data
np.random.shuffle(X_train)
np.random.shuffle(X_test)

# Slice out only the first 141 from X_train and 213 from X_test
X_train = X_train[0:141]
X_test = X_test[0:213]

# Remove the last columns of X_train and X_test and place them back into y_train and y_test
y_train = X_train[:,-1]
y_test = X_test[:,-1]
X_train = X_train[:,0:X_train.shape[1]-1]
X_test = X_test[:,0:X_test.shape[1]-1]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(141, 784)
(141,)
(213, 784)
(213,)


In [30]:
# Scale the dataset

X_scale = StandardScaler()
X_train = X_scale.fit_transform(X_train) 
X_test = X_scale.fit_transform(X_test) 

In [31]:
def kernel_svm(X, y): 

    m,n = X.shape
    y = y.reshape(-1,1)*1.
    X_y = y*X
    H = np.dot(X_y, X_y.T)*1.

    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    G = matrix(-np.eye(m))
    h = matrix(np.zeros(m))
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))

    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_svm(X_train, y_train)
print(alphas)

     pcost       dcost       gap    pres   dres
 0: -7.5000e+01 -1.5000e+02  3e+02  1e+01  2e+00
 1: -3.3583e+02 -3.3919e+02  7e+01  6e+00  1e+00
 2: -3.7729e+04 -3.7733e+04  7e+01  6e+00  1e+00
 3: -3.7397e+08 -3.7397e+08  4e+02  6e+00  1e+00
 4: -3.7023e+14 -3.7023e+14  4e+06  6e+00  1e+00
 5: -3.6653e+22 -3.6653e+22  4e+12  4e+00  1e+00
 6: -3.6286e+32 -3.6286e+32  4e+20  5e+15  1e+00
 7: -3.5918e+44 -3.5918e+44  4e+30  5e+27  1e+00
 8: -3.5881e+58 -3.5881e+58  4e+42  8e+41  1e+00
 9: -2.0543e+74 -2.0543e+74  2e+56  4e+00  1e+00
10: -7.5601e+91 -7.5601e+91  8e+71  4e+75  1e+00
11: -1.8306e+108 -1.8306e+108  2e+86  4e+91  1e+00
12: -1.8123e+152 -3.9306e+157  4e+157 6e+135  2e+05
13: -1.8123e+152 -3.9324e+155  4e+155 3e+135  2e+03
14: -1.8123e+152 -3.9324e+155  4e+155 3e+135  2e+03
15: -1.8123e+152 -3.9324e+155  4e+155 3e+135  2e+03
16: -1.8123e+152 -3.9324e+155  4e+155 3e+135  2e+03
17: -1.8123e+152 -3.9324e+155  4e+155 3e+135  2e+03
18: -1.8123e+152 -3.9324e+155  4e+155 3e+135  2e+0

In [32]:
def compute_classification_boundary (X, y, alpha):
    #cond = (alphas > 1e-3).reshape(-1)
    cond = [i for i in range(len(alphas)) if alphas[i] < 1e-7]
    w = np.dot(X.T, alpha*y).reshape(-1,1)
    w0 = y[cond] - np.dot(X[cond], w)
    w0 = np.mean(w0)
    return w, w0



w, w0 = compute_classification_boundary(X_train, y_train, alphas)

In [33]:
# Determine which training examples are support vectors
support_vectors = []

for i in range(len(alphas)):
    if alphas[i] < 1e-7:
        support_vectors.append([X_train[i], y_train[i], i])

In [34]:
def K(xi, xj):
    return np.dot(xi,xj)

alpha_indices = [support_vectors[i][2] for i in range(len(support_vectors))]
print(alpha_indices)

[2, 9, 15, 16, 19, 25, 28, 30, 32, 34, 36, 37, 43, 44, 46, 48, 49, 51, 53, 56, 57, 58, 66, 68, 74, 75, 77, 79, 80, 83, 85, 87, 93, 94, 95, 96, 100, 101, 102, 103, 106, 110, 111, 115, 120, 122, 123, 124, 125, 127, 129, 130, 132, 133, 134, 136]


In [35]:
def f_dual(x):
    summation = 0
    for i in range(len(support_vectors)):
        summation += alphas[alpha_indices[i]]*y_train[alpha_indices[i]]*K(X_train[alpha_indices[i]],x)
    if (summation >= 0):
        return 0
    else:
        return 2

In [36]:
# Test SVM dual classifier on X_test

def predict(X):
    predictions = []
    for i in range(len(X_test)):
        predictions.append(f_dual(X_test[i]))
    return predictions

y_pred = predict(X_test)

In [37]:
# Print accuracy

print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy is 94.83568075117371%


### Extended implementation using Numpy

In [38]:
def kernel_soft_margin_svm(X, y, C): 

    m,n = X.shape
    y = y.reshape(-1,1)*1.
    X_y = X*y
    H = np.dot(X_y, X_y.T)*1.
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    
    # Changed G and h
    G = matrix(np.vstack((np.diag(np.ones(m))*-1, np.identity(m))))
    h = matrix(np.hstack((np.zeros(m), np.ones(m)*C)))
    
    A = matrix(y.reshape(1,-1))
    A = matrix(A, (1, m), 'd')
    b = matrix(np.zeros(1))
    
    sol = solvers.qp(P,q,G,h,A,b) 
    
    alphas = np.array(sol['x'])[:,0]
    
    return alphas

# fit svm dual classifier
alphas = kernel_soft_margin_svm(X_train, y_train, 0.01)
print(alphas)

     pcost       dcost       gap    pres   dres
 0: -3.7875e+01 -2.4867e+00  6e+02  3e+01  3e-16
 1: -9.7299e-01 -2.4678e+00  8e+00  3e-01  2e-16
 2: -6.3553e-01 -1.4803e+00  9e-01  3e-03  3e-16
 3: -7.3085e-01 -7.7093e-01  4e-02  1e-04  5e-16
 4: -7.4981e-01 -7.5021e-01  4e-04  1e-06  3e-16
 5: -7.5000e-01 -7.5000e-01  4e-06  1e-08  3e-16
 6: -7.5000e-01 -7.5000e-01  4e-08  1e-10  2e-16
Optimal solution found.
[ 9.99999974e-03  9.99999974e-03  5.68122494e-26  9.99999974e-03
  9.99999974e-03  9.99999974e-03  9.99999974e-03  9.99999974e-03
  9.99999974e-03 -1.98842391e-26  9.99999974e-03 -3.45416816e-26
  9.99999974e-03  9.99999974e-03  9.99999974e-03 -1.35688632e-25
 -2.25866111e-26  9.99999974e-03  9.99999974e-03 -2.09652939e-25
  9.99999974e-03  9.99999974e-03  9.99999974e-03  9.99999974e-03
  9.99999974e-03 -5.44459569e-26  9.99999974e-03  9.99999974e-03
 -8.67804406e-26  9.99999974e-03  4.29290755e-26  9.99999974e-03
  2.85915634e-25  9.99999974e-03  2.21012742e-25  9.99999974e-03


In [39]:
def compute_classification_boundary (X, y, alpha):
    #cond = (alphas > 1e-3).reshape(-1)
    cond = [i for i in range(len(alphas)) if alphas[i] < 0]
    w = np.dot(X.T, alpha*y).reshape(-1,1)
    w0 = y[cond] - np.dot(X[cond], w)
    w0 = np.mean(w0)
    return w, w0



w, w0 = compute_classification_boundary(X_train, y_train, alphas)

In [40]:
# Determine which training examples are support vectors
support_vectors = []

for i in range(len(alphas)):
    if alphas[i] < 0:
        support_vectors.append([X_train[i], y_train[i], i])

# print("The following are support vectors: ")
# for i in range(len(support_vectors)):
#     print(support_vectors[i][0])

In [41]:
def K(xi, xj):
    return np.dot(xi,xj)

alpha_indices = [support_vectors[i][2] for i in range(len(support_vectors))]
print(alpha_indices)

[9, 11, 15, 16, 19, 25, 28, 36, 43, 44, 46, 48, 49, 51, 53, 56, 57, 58, 68, 74, 77, 79, 80, 83, 91, 95, 101, 103, 105, 106, 111, 124, 127, 130, 132]


In [42]:
def f_dual(x):
    summation = 0
    for i in range(len(support_vectors)):
        summation += alphas[alpha_indices[i]]*y_train[alpha_indices[i]]*K(X_train[alpha_indices[i]],x)
    if (summation >= 0):
        return 0
    else:
        return 2

In [43]:
# Test SVM dual classifier on X_test

def predict(X):
    predictions = []
    for i in range(len(X_test)):
        predictions.append(f_dual(X_test[i]))
    return predictions

y_pred = predict(X_test)

In [44]:
# Print accuracy

print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))

Prediction accuracy is 93.42723004694837%
